In [1]:
import ee
import pandas as pd
print("Earth Engine API imported successfully!")

Earth Engine API imported successfully!


In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-damyadharmawardhana')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


### Collect rainfall and Temp data from GEE (Daily wise)

In [20]:
# Define the area of interest (ROI)
aoi = ee.Geometry.Polygon([
    [[79.80086, 7.62606], [79.86712, 7.62470],
     [79.86712, 7.59595], [79.78592, 7.59731], [79.80086, 7.62606]]
])

# Define the time range
start_date = '1980-01-01'
end_date = '2024-12-31'

# Load the ERA5 Daily dataset
era5 = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .filterDate(start_date, end_date) \
    .select('mean_2m_air_temperature')

# Function to reduce the image collection to mean temperature and date
def extract_temperature(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    temperature = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000,
        maxPixels=1e9
    ).get('mean_2m_air_temperature')
    return ee.Feature(None, {'date': date, 'temperature': temperature})

# Apply the function to the image collection
temperature_series = era5.map(extract_temperature)

# Export the data to a CSV file in Google Drive
task = ee.batch.Export.table.toDrive(
    collection=temperature_series,
    description='Temperature_Data_Export_new',
    fileFormat='CSV',
    folder='GEE_Temperature'  # You can choose your Google Drive folder here
)
task.start()

print("Export started. Check your Google Drive after processing.")

Export started. Check your Google Drive after processing.


In [16]:
# Define the area of interest (ROI)
aoi = ee.Geometry.Polygon([
    [[79.80086, 7.62606], [79.86712, 7.62470],
     [79.86712, 7.59595], [79.78592, 7.59731], [79.80086, 7.62606]]
])

# Define the time range
start_date = '1980-01-01'
end_date = '2024-12-31'

# Load the ERA5 Daily dataset
era5 = ee.ImageCollection('ECMWF/ERA5/DAILY') \
    .filterDate(start_date, end_date)

# Function to reduce the image collection to mean rainfall and date
def extract_rainfall(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    rainfall = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000,
        maxPixels=1e9
    ).get('total_precipitation')  # Adjust to the correct rainfall band
    return ee.Feature(None, {'date': date, 'rainfall': rainfall})

# Select the rainfall band and apply the function
rainfall_series = era5.select('total_precipitation').map(extract_rainfall)

# Export the data to a CSV file in Google Drive
task = ee.batch.Export.table.toDrive(
    collection=rainfall_series,
    description='Rainfall_Data_Export',
    fileFormat='CSV',
    folder='GEE_Rainfall'  # You can choose your Google Drive folder here
)
task.start()

print("Rainfall export started. Check your Google Drive after processing.")

Rainfall export started. Check your Google Drive after processing.


In [17]:
Temp_data = pd.read_csv('Temperature_Data_Export.csv')

In [18]:
Temp_data

,system:index,date,temperature,.geo
0,19880101,1988-01-01,299.096527,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,19880102,1988-01-02,299.288574,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,19880103,1988-01-03,299.616119,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,19880104,1988-01-04,299.900421,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,19880105,1988-01-05,299.642944,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
11874,20200705,2020-07-05,301.647858,"{""type"":""MultiPoint"",""coordinates"":[]}"
11875,20200706,2020-07-06,300.987274,"{""type"":""MultiPoint"",""coordinates"":[]}"
11876,20200707,2020-07-07,301.741394,"{""type"":""MultiPoint"",""coordinates"":[]}"
11877,20200708,2020-07-08,301.607697,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [21]:
rainfall_data = pd.read_csv('Rainfall_Data_Export.csv')

In [22]:
rainfall_data

,system:index,date,rainfall,.geo
0,19880101,1988-01-01,4.284084e-08,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,19880102,1988-01-02,1.137517e-05,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,19880103,1988-01-03,1.071952e-06,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,19880104,1988-01-04,2.184324e-05,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,19880105,1988-01-05,2.148878e-04,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...
11874,20200705,2020-07-05,1.171062e-03,"{""type"":""MultiPoint"",""coordinates"":[]}"
11875,20200706,2020-07-06,1.046159e-02,"{""type"":""MultiPoint"",""coordinates"":[]}"
11876,20200707,2020-07-07,1.210762e-03,"{""type"":""MultiPoint"",""coordinates"":[]}"
11877,20200708,2020-07-08,1.052359e-03,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [6]:
#merge rainfall and temp data

temperature_data = pd.read_csv('Temperature_Data_Export.csv')  # File with date and temperature
rainfall_data = pd.read_csv('Rainfall_Data_Export.csv')  # File with date and rainfall

# Ensure both datasets have 'date' in the same format
temperature_data['date'] = pd.to_datetime(temperature_data['date'])
rainfall_data['date'] = pd.to_datetime(rainfall_data['date'])

# Merge the two datasets on the 'date' column
temp_rainfall_data = pd.merge(temperature_data, rainfall_data, on='date', how='inner')

In [7]:
temp_rainfall_data

,system:index_x,date,temperature,.geo_x,system:index_y,rainfall,.geo_y
0,19800101,1980-01-01,299.401581,"{""type"":""MultiPoint"",""coordinates"":[]}",19800101,0.000313,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,19800102,1980-01-02,298.765991,"{""type"":""MultiPoint"",""coordinates"":[]}",19800102,0.000000,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,19800103,1980-01-03,298.795135,"{""type"":""MultiPoint"",""coordinates"":[]}",19800103,0.000000,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,19800104,1980-01-04,299.181213,"{""type"":""MultiPoint"",""coordinates"":[]}",19800104,0.000420,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,19800105,1980-01-05,299.083893,"{""type"":""MultiPoint"",""coordinates"":[]}",19800105,0.000914,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...,...,...
14796,20200705,2020-07-05,301.647858,"{""type"":""MultiPoint"",""coordinates"":[]}",20200705,0.001171,"{""type"":""MultiPoint"",""coordinates"":[]}"
14797,20200706,2020-07-06,300.987274,"{""type"":""MultiPoint"",""coordinates"":[]}",20200706,0.010462,"{""type"":""MultiPoint"",""coordinates"":[]}"
14798,20200707,2020-07-07,301.741394,"{""type"":""MultiPoint"",""coordinates"":[]}",20200707,0.001211,"{""type"":""MultiPoint"",""coordinates"":[]}"
14799,20200708,2020-07-08,301.607697,"{""type"":""MultiPoint"",""coordinates"":[]}",20200708,0.001052,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [8]:
#remove unncessary rows

temp_rainfall_data = temp_rainfall_data.drop(['.geo_x', '.geo_y', 'system:index_y', 'system:index_x'], axis=1)

In [9]:
temp_rainfall_data

,date,temperature,rainfall
0,1980-01-01,299.401581,0.000313
1,1980-01-02,298.765991,0.000000
2,1980-01-03,298.795135,0.000000
3,1980-01-04,299.181213,0.000420
4,1980-01-05,299.083893,0.000914
...,...,...,...
14796,2020-07-05,301.647858,0.001171
14797,2020-07-06,300.987274,0.010462
14798,2020-07-07,301.741394,0.001211
14799,2020-07-08,301.607697,0.001052
